## Regular Expressions

That said, learning (and loving!) regular expressions is something that is a worthwhile investment
<ul>
<li>Once you understand how they work, complex operations with string data can be written a lot quicker, which will save you time.</li>
    <li>Regular expressions are often faster to execute than their manual equivalents.</li>
<li>Regular expressions are supported in almost every modern programming language, as well as other places like command line utilities and databases. Understanding regular expressions gives you a powerful tool that you can use wherever you work with data.</li>
    </ul>

The dataset we will be working with is based off this CSV of Hacker News stories from September 2015 to September 2016. The columns in the dataset are explained below:
<ul>
    <li><b>id</b>: The unique identifier from Hacker News for the story</li>
    <li><b>title</b>: The title of the story</li>
    <li><b>url</b>: The URL that the stories links to, if the story has a URL</li>
    <li><b>num_points</b>: The number of points the story acquired, calculated as the total number of upvotes minus the total number of downvotes</li>
    <li><b>num_comments</b>: The number of comments that were made on the story</li>
    <li><b>author</b>: The username of the person who submitted the story</li>
    <li><b>created_at</b>: The date and time at which the story was submitted</li>
    </ul>

For teaching purposes, the dataset has been reduced from the almost 300,000 rows in its original form to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. You can download the modified dataset using the dataset preview tool.

When working with regular expressions, we use the term <b>pattern</b> to describe a regular expression that we've written. If the pattern is found within the string we're searching, we say that it has matched.

As we previously learned, letters and numbers represent themselves in regular expressions. If we wanted to find the string "and" within another string, the regex pattern for that is simply and:
![ ](regular1.png)

<br>The first of these we'll learn is called a set. A set allows us to specify two or more characters that can match in a single character's position.</br>

<br>We define a set by placing the characters we want to match for in square brackets:</br>
![ ](regular2.png)

The regular expression above will match the strings <b>mend</b>, <b>send</b>, and <b>bend</b>.

Let's look at how we can add sets to match more of our example strings from earlier:
![ ](regular3.png)